# Imports

In [2]:
from utils import *
import os
import shutil
from PIL import Image
from pytorch_fid import fid_score
import copy

import torch.distributions as D

import torch.optim.lr_scheduler as lrsc
import wandb
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

from config import *
config = get_configs()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.login()
train_loader, val_loader = get_dataset_CIFAR10(config)

Files already downloaded and verified
Files already downloaded and verified


# Exp

In [3]:
# eps_th = nn.DataParallel(Unet(config))
eps_th = Unet(config)
eps_th.to(device)

optim = torch.optim.Adam(eps_th.parameters(), lr=2e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
ema_ = ema.ExponentialMovingAverage(eps_th.parameters(), decay=0.9999)

In [ ]:
wandb.init(project='upsample_cifar')
train(eps_th, train_loader, optim, ema_, 2688, 1, 0.0, device, config)

  0%|          | 0/2688 [00:00<?, ?it/s]

wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
wandb: ERROR Error while calling W&B API: Error 1040: Too many connections (<Response [500]>)
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
